In [21]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np

In [14]:
fch = pd.read_csv("data/FINAL_FEATURES_TRAINTEST.tsv",sep='\t')
trg = pd.read_csv("data/FINAL_TARGETS_DATES_TRAINTEST.tsv",sep="\t")

In [15]:
new_fch_trg = pd.merge(trg, fch, on="CLIENT_ID",how="left")

In [16]:
new_fch_trg.drop(["CLIENT_ID"],axis=1,inplace=True)
new_fch_trg.drop(["RETRO_DT"],axis=1,inplace=True)

In [17]:
new_fch_trg.fillna(0)
new_test = new_fch_trg[(new_fch_trg["TARGET"]=='test')]
new_fch_trg.drop(new_fch_trg.index[(new_fch_trg["TARGET"]=='test')],axis=0,inplace=True)
new_fch = new_fch_trg.drop(["TARGET"],axis=1)
new_trg = new_fch_trg["TARGET"]

In [86]:
cat = CatBoostClassifier()
cat.fit(X=new_fch,y=new_trg,verbose=False)

In [87]:
new_test_fch = new_test.drop(["TARGET"],axis=1)
new_test_fch

,i1047,i1048,i1056,i1058,i1059,i1060,i1065,i1066,i1067,i1068,...,u8=17,u8=18,u8=19,u8=2,u8=20,u8=21,u8=6,u8=7,u8=8,u8=9
4,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159263,0.0,6.0,14.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
159276,0.0,10.0,32.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159284,0.0,0.0,82.0,6.0,0.0,11.0,28.0,0.0,32.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
pred = cat.predict_proba(new_fch,verbose=False)
pred[::1]

array([[0.79106338, 0.20893662],
       [0.54568832, 0.45431168],
       [0.87488677, 0.12511323],
       ...,
       [0.59036048, 0.40963952],
       [0.65200334, 0.34799666],
       [0.745122  , 0.254878  ]])

In [19]:
roc_auc_score(new_trg,pred[:, 1])

0.7162899260999108

In [76]:
join_fch_trg = pd.merge(trg, fch, on="CLIENT_ID",how="left")
join_fch_trg.drop(["CLIENT_ID"],axis=1,inplace=True)
join_fch_trg.drop(["RETRO_DT"],axis=1,inplace=True)
join_fch_trg.drop(join_fch_trg.index[(join_fch_trg["TARGET"]=='test')],axis=0,inplace=True)
join_fch_trg.fillna(0,inplace=True)
#join_fch_trg = join_fch_trg.reset_index()
mask = np.random.choice(join_fch_trg.shape[0], size=join_fch_trg.shape[0], replace=False)
ratio = int(0.8 * len(mask))
train_dataset, val_dataset = join_fch_trg.iloc[list(mask[:ratio])], join_fch_trg.iloc[list(mask[ratio:])]

In [79]:
train_fch = train_dataset.drop(["TARGET"],axis=1)
train_trg = train_dataset["TARGET"]
cat.fit(X=train_fch,y=train_trg,verbose=False)

In [83]:
val_fch = val_dataset.drop(["TARGET"],axis=1)
val_trg = val_dataset["TARGET"]
new_prd = cat.predict_proba(val_fch,verbose=False)

In [85]:
roc_auc_score(val_trg,new_prd[:,1])

0.5731665720349775

In [90]:
submission = pd.read_csv("sample_submission.csv")

In [93]:
features = pd.merge(submission, fch, on="CLIENT_ID", how="left")
X = features.drop("CLIENT_ID", axis=1)
submission["TARGET"] = cat.predict_proba(X)[:, 1]
submission

,CLIENT_ID,TARGET
0,1504364,0.430327
1,1592092,0.240887
2,1582159,0.240887
3,1431753,0.242886
4,1571196,0.240887
...,...,...
31853,1518155,0.236186
31854,1544546,0.189165
31855,1584674,0.240887
31856,1733854,0.240887


In [94]:
submission.to_csv("submission.csv", index=False)